# Imports

In [2]:
import sys
sys.path.append('/home/beav3r/.local/lib/python3.10/site-packages')

In [15]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

# from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score
# import graphviz
# import pydotplus

from xgboost import XGBClassifier
import lightgbm as lgb
# from catboost import CatBoostClassifier

sns.set(style="darkgrid")

# Data

In [4]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train.head()

,session_id,webpage1,time1,webpage2,time2,webpage3,time3,webpage4,time4,webpage5,...,time6,webpage7,time7,webpage8,time8,webpage9,time9,webpage10,time10,target
0,0,9486,2019-02-20 05:57:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,1,11722,2019-02-22 07:14:50,12385.0,2019-02-22 07:14:50,50163.0,2019-02-22 07:14:51,12385.0,2019-02-22 07:14:51,12398.0,...,2019-02-22 07:14:51,50163.0,2019-02-22 07:14:52,50150.0,2019-02-22 07:14:52,19860.0,2019-02-22 07:15:15,19886.0,2019-02-22 07:15:16,0
2,2,192149,2018-12-16 12:35:17,659.0,2018-12-16 12:35:18,192136.0,2018-12-16 12:35:19,192149.0,2018-12-16 12:35:19,633.0,...,2018-12-16 12:35:19,192136.0,2018-12-16 12:35:20,192136.0,2018-12-16 12:35:21,192136.0,2018-12-16 12:35:22,192136.0,2018-12-16 12:35:24,0
3,3,10591,2019-02-13 12:40:35,451.0,2019-02-13 12:40:35,77580.0,2019-02-13 12:40:35,227821.0,2019-02-13 12:40:35,633.0,...,2019-02-13 12:42:14,10591.0,2019-02-13 12:42:14,227834.0,2019-02-13 12:42:15,227834.0,2019-02-13 12:42:16,227834.0,2019-02-13 12:42:17,0
4,4,438,2018-04-12 06:22:26,425.0,2018-04-12 06:22:26,529.0,2018-04-12 06:22:28,65685.0,2018-04-12 06:22:29,187638.0,...,2018-04-12 06:22:29,425.0,2018-04-12 06:22:29,65685.0,2018-04-12 06:22:31,187625.0,2018-04-12 06:22:31,187625.0,2018-04-12 06:22:32,0


# EDA

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159969 entries, 0 to 159968
Data columns (total 22 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   session_id  159969 non-null  int64  
 1   webpage1    159969 non-null  int64  
 2   time1       159969 non-null  object 
 3   webpage2    157990 non-null  float64
 4   time2       157990 non-null  object 
 5   webpage3    156066 non-null  float64
 6   time3       156066 non-null  object 
 7   webpage4    154402 non-null  float64
 8   time4       154402 non-null  object 
 9   webpage5    152841 non-null  float64
 10  time5       152841 non-null  object 
 11  webpage6    151324 non-null  float64
 12  time6       151324 non-null  object 
 13  webpage7    149912 non-null  float64
 14  time7       149912 non-null  object 
 15  webpage8    148574 non-null  float64
 16  time8       148574 non-null  object 
 17  webpage9    147160 non-null  float64
 18  time9       147160 non-null  object 
 19  we

In [6]:
"""Considering that our columns categorical and timestamps, I am going to use LightGBM"""
train.describe()

,session_id,webpage1,webpage2,webpage3,webpage4,webpage5,webpage6,webpage7,webpage8,webpage9,webpage10,target
count,159969.000000,159969.000000,157990.000000,156066.000000,154402.000000,152841.000000,151324.000000,149912.000000,148574.000000,147160.000000,145856.000000,159969.000000
mean,79984.000000,44362.540573,44593.501722,45025.879231,45448.173081,45663.769584,45820.414706,46410.833809,46813.332925,46971.420053,47558.884736,0.009846
std,46179.216943,97167.414738,97682.859444,97995.435761,98488.571326,98504.193709,98600.684290,99314.768320,99651.347793,99794.332873,100488.244994,0.098736
min,0.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,165.000000,0.000000
25%,39992.000000,828.000000,828.000000,828.000000,828.000000,828.000000,828.000000,828.000000,828.000000,828.000000,828.000000,0.000000
50%,79984.000000,8160.000000,8160.000000,8173.000000,8238.000000,8693.000000,8810.000000,8940.000000,8953.000000,8953.000000,8953.000000,0.000000
75%,119976.000000,30845.000000,30663.000000,31508.000000,31716.000000,31716.000000,31963.000000,32704.000000,33809.000000,33835.000000,34108.000000,0.000000
max,159968.000000,540965.000000,540952.000000,540939.000000,540939.000000,540939.000000,540952.000000,540952.000000,540965.000000,540965.000000,540965.000000,1.000000


In [6]:
"""Quite a lot for one hot encoding"""
train["webpage1"].unique().shape

(11565,)

# Data preparation

In [7]:
# Convert timestamps to datetime format
for i in range(1, 11):
    train[f'time{i}'] = pd.to_datetime(train[f'time{i}'], errors='coerce')
    test[f'time{i}'] = pd.to_datetime(test[f'time{i}'], errors='coerce')

# Feature Engineering: Calculate time differences between consecutive pages
for i in range(1, 10):
    train[f'time_diff_{i}'] = (train[f'time{i+1}'] - train[f'time{i}']).dt.total_seconds()
    test[f'time_diff_{i}'] = (test[f'time{i+1}'] - test[f'time{i}']).dt.total_seconds()

# Replace NaN time differences with 0 (for sessions with fewer than 10 transitions)
train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

# Drop the original timestamps
time_columns = [f'time{i}' for i in range(1, 11)]
train.drop(columns=time_columns, inplace=True)
test.drop(columns=time_columns, inplace=True)

# # Convert webpage IDs to strings (No need)
# webpage_columns = [f'webpage{i}' for i in range(1, 11)]
# train[webpage_columns] = train[webpage_columns].astype(str)
# test[webpage_columns] = test[webpage_columns].astype(str)

df_train = train.copy()
df_test = test.copy()

"""
# One-hot encode the webpage columns (So many categories)
df_train = pd.get_dummies(train, columns=webpage_columns, drop_first=True)
df_test = pd.get_dummies(test, columns=webpage_columns, drop_first=True)
"""

# Separate features and target
X = df_train.drop(columns=['session_id', 'target'])
X_test = df_test.drop(columns='session_id')
y = df_train['target']

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True, stratify=y)

/tmp/ipykernel_708715/457391306.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  train.fillna(0, inplace=True)
/tmp/ipykernel_708715/457391306.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  test.fillna(0, inplace=True)


# LightGBM

In [8]:
# Convert the dataset into LightGBM Dataset format
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

# Define LightGBM parameters
params = {
    'objective': 'binary',
    'metric': 'auc',
    'boosting_type': 'gbdt',
    'num_leaves': 19,
    'learning_rate': 0.03,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1
}

# Train the model
lgb_model = lgb.train(params,
                      train_data,
                      num_boost_round=10000,
                      valid_sets=[train_data, valid_data],
                      callbacks=[lgb.early_stopping(stopping_rounds=200), lgb.log_evaluation(period=50)])

# Predict on the validation set
y_pred_proba = lgb_model.predict(X_val, num_iteration=lgb_model.best_iteration)

# Evaluate the model
roc_auc = roc_auc_score(y_val, y_pred_proba)
print(f'ROC-AUC: {roc_auc:.4f}')

Training until validation scores don't improve for 200 rounds
[50]	training's auc: 0.898178	valid_1's auc: 0.858855
[100]	training's auc: 0.939081	valid_1's auc: 0.888585
[150]	training's auc: 0.957609	valid_1's auc: 0.894896
[200]	training's auc: 0.966559	valid_1's auc: 0.898132
[250]	training's auc: 0.973218	valid_1's auc: 0.903921
[300]	training's auc: 0.97853	valid_1's auc: 0.907394
[350]	training's auc: 0.983764	valid_1's auc: 0.909902
[400]	training's auc: 0.987959	valid_1's auc: 0.911346
[450]	training's auc: 0.98965	valid_1's auc: 0.913173
[500]	training's auc: 0.991608	valid_1's auc: 0.913797
[550]	training's auc: 0.99283	valid_1's auc: 0.914114
[600]	training's auc: 0.993769	valid_1's auc: 0.915083
[650]	training's auc: 0.994983	valid_1's auc: 0.914662
[700]	training's auc: 0.995565	valid_1's auc: 0.913954
[750]	training's auc: 0.996204	valid_1's auc: 0.914364
Early stopping, best iteration is:
[591]	training's auc: 0.99361	valid_1's auc: 0.915473
ROC-AUC: 0.9155


In [ ]:
# Computing probabilities for the test set
y_pred_proba_test = lgb_model.predict(X_test, num_iteration=lgb_model.best_iteration)

# Prepare the submission dataframe
submission_df = pd.DataFrame({
    'session_id': df_test['session_id'],  # or use df_test['session_id'] if using a test set
    'target': y_pred_proba_test
})

# Ensure the correct format
submission_df['target'] = submission_df['target'].astype(float)

# Save the submission file
submission_df.to_csv('submission_LightGBM.csv', index=False)

# XGBoost

In [12]:
# Define the XGBoost model
xgb_model = XGBClassifier(n_estimators=10000,
                          learning_rate=0.03,
                          max_depth=21,
                          subsample=0.8,
                          colsample_bytree=0.8,
                          objective='binary:logistic',
                          random_state=42,
                          tree_method='gpu_hist',
                          eval_metric='auc',
                          early_stopping_rounds = 200)

# Train the model
xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=100)

# Predict on the train set
y_pred_proba = xgb_model.predict_proba(X_test)[:, 1]

# Prepare the submission dataframe
submission_df = pd.DataFrame({
    'session_id': df_test['session_id'],  # or use df_test['session_id'] if using a test set
    'target': y_pred_proba
})

# Ensure the correct format
submission_df['target'] = submission_df['target'].astype(float)

# Save the submission file
submission_df.to_csv('submission_XGboost.csv', index=False)

[0]	validation_0-auc:0.72121


/home/beav3r/myCourseswsl/EPAM_AI_Course_0824_wsl/myenv_09082024/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [12:17:38] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


[100]	validation_0-auc:0.89440
[200]	validation_0-auc:0.90929
[300]	validation_0-auc:0.90894
[400]	validation_0-auc:0.90555
[432]	validation_0-auc:0.90488


/home/beav3r/myCourseswsl/EPAM_AI_Course_0824_wsl/myenv_09082024/lib/python3.10/site-packages/xgboost/core.py:158: UserWarning: [12:17:47] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
